In [16]:
import numpy as np
import cv2 as cv
import glob
import math

In [4]:

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('*.png')


In [5]:
for fname in images:
 img = cv.imread(fname)
 gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
 # Find the chess board corners
 ret, corners = cv.findChessboardCornersSB(gray, (7,7), None)


 print(ret)
 
 # If found, add object points, image points (after refining them)
 if ret == True:
    objpoints.append(objp)
 else:
   continue
 
 corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
 imgpoints.append(corners2)
 
 # Draw and display the corners
 cv.drawChessboardCorners(img, (7,7), corners, ret)

 cv.imshow('img', img)
 cv.waitKey(500)
 
cv.destroyAllWindows()

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [6]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
img = cv.imread(r'New_folder/test.png')
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [7]:
print(newcameramtx)
print(dist)
print(rvecs)
print(tvecs)


[[553.05377197   0.         959.03121998]
 [  0.         522.69873047 538.21025506]
 [  0.           0.           1.        ]]
[[-3.52150447e-01  1.66373267e-01 -6.63122757e-05  5.06891510e-06
  -4.45939058e-02]]
(array([[-0.09437708],
       [ 0.01593641],
       [-0.04335968]]), array([[-0.09428622],
       [ 0.01613284],
       [-0.04335218]]), array([[-0.0941953 ],
       [ 0.01600114],
       [-0.04332838]]), array([[-0.09428087],
       [ 0.0159561 ],
       [-0.04334418]]), array([[-0.0942004 ],
       [ 0.01575061],
       [-0.04334702]]), array([[-0.09376254],
       [ 0.01616333],
       [-0.04338526]]), array([[-0.0940495 ],
       [ 0.01600914],
       [-0.04337416]]), array([[-0.09395687],
       [ 0.01609038],
       [-0.04335836]]), array([[-0.09414618],
       [ 0.01622633],
       [-0.0433707 ]]), array([[-0.09414489],
       [ 0.01587005],
       [-0.0433239 ]]), array([[-0.09408129],
       [ 0.01612614],
       [-0.04335777]]), array([[-0.0942138 ],
       [ 0.01642

In [8]:
gray.shape[::-1]
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('New folder\calibresult.png', dst)
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('New folder\calibresult_rect.png', dst)


False

In [9]:
cv.imshow('d',dst)
cv.waitKey(0)

-1

---
### Replicating the Leopard camera model *LI-VENUS-OX03D4C-GM2A-124H* witha field of view of 124 degrees horizontal and reslution of 1920 (H) x 1536 (V)


In [23]:
camera_matrix_124= newcameramtx
camera_matrix_124


array([[553.05377197,   0.        , 959.03121998],
       [  0.        , 522.69873047, 538.21025506],
       [  0.        ,   0.        ,   1.        ]])

In [26]:
def calculate_d(focal_length, fov):
    fov_radians = math.radians(fov)
    d = 2 * focal_length * math.tan(fov_radians / 2)
    return d



fx=newcameramtx[0][0]
fy=newcameramtx[1][1]
given_fov= 120 #degrees

dx= calculate_d(fx, given_fov)
dy= calculate_d(fy, given_fov)

print(dx,dy)




1915.8344647485053 1810.6815164472503


In [27]:
#calculate the field of view in vertical    direction

FOV_x=124
h=1920 #horizontal resolution
v=1536 #vertical resoltion
aspect_ratio= h/v
FOV_y= FOV_x/aspect_ratio
FOV_y


99.2

In [25]:
#calculation of new focal lengths


def calculate_focal_len(d,fov):
    fov_radians = math.radians(fov)
    f=d/(2*math.tan(fov_radians/2))
    return(f)

fov_x =124
fov_y= 99.2
fx_new = calculate_focal_len(dx,fov_x)
fy_new = calculate_focal_len(dy,fov_y)
print(fx_new,fy_new)




509.3336272044506 770.5053794946618
